<div class='alert alert-danger'>
<b>Косяки:</b>

1. Сравнение таблиц не работает достаточно гибко для датафреймов с разными количествами столбцов
2. Не осуществлен автоматический перебор вариантов для `build_prompt`
</div>

<div class='alert alert-warning'>
<b>Нюансы:</b>

1. Для тестирования моделей необходимы два объекта: непосредственно соединение с базой данных и таблица с запросами к этим данным.
Таблица с запросами должна удовлетворять одному условию - она должна состоять из столбцов с названиями 'question' и 'query'.
К базе данных строгих требований нет.

2. Существуют, по крайней мере, два модуля в питоне, которые предоставляют интерфейс взаимодействия с базами данных SQlite -- sqlite3 и sqlalchemy. 
Мы будем пользоваться модулем sqlalchemy по той простой причине, что он позволяет напрямую читать .xlsx таблицы как SQlite базу данных. Важно, что в библиотеке
sqlite3, чтобы сделать запрос в бд, надо написать строку вида `conn.execute(query)`, где query - str. В sqlalchemy немного иначе - `conn.execute(text(query))`;
функция text лежит в этом же модуле. 
</div>

## Загрузка необходимых модулей и датасета

In [ ]:
!pip install json5 gdown sentence-transformers -q

In [ ]:
from tqdm import tqdm
import pandas as pd
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np
from utils.general import *
import torch

from sqlglot import parse_one
from sqlglot.diff import ChangeDistiller
from spans import *

from sqlalchemy import create_engine
from prompting import PromptBuilder
from sklearn.utils import shuffle
from sqlalchemy import Connection
from utils.dataset import *

In [ ]:
engine = create_engine('sqlite:///main_database.sqlite', echo=False)
conn = engine.connect()

In [ ]:
prepare_column_names(conn) # Устраняет пробелы в названии столбцов
queries = IterableDataFrame(pd.read_excel('NLSQL.xlsx'))

# Препроцессинг промпта

In [ ]:
sentence_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
class HuggingFaceModelInference:
    def __init__(self, path):
        self.path = path
        self.evaluated = False
        self.is_downloaded = False


    def __load_model(self):
        self.tokenizer = AutoTokenizer.from_pretrained(self.path, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(
                    self.path,
                    torch_dtype=torch.float16,
                    device_map="auto",
                    max_memory={0: "10GiB", 1: "10GiB"},  
                    offload_folder="./offload", 
                    trust_remote_code=True
                    )

    def __inference(self, prompt):
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        with torch.inference_mode():  
            inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device) 
            generate_ids = self.model.generate(
                            **inputs,
                            max_length=2048,
                            num_return_sequences=1,
                            temperature=0.2, 
                            top_p=0.95,
                            do_sample=True,
                            use_cache=True 
                            )
    
            output = self.tokenizer.decode(
                    generate_ids[0, inputs.input_ids.shape[1]:],
                    skip_special_tokens=True
                    )
            
        return output
    

    def evaluate(self, queries : IterableDataFrame, connection : Connection):
        if not self.is_downloaded:
            self.__load_model()
            self.is_downloaded = True

        self.model.eval()

        logger : list[ExtendedSqlSpan] = []
        summary = 0
        for query in tqdm(queries):
            question = query['question']
            gold_sql = query['query']

            prompt = PromptBuilder()\
                .add_message('### You are an expert SQL developer with deep knowledge of database optimization, correct syntax, and efficient query design. Your task is to generate accurate, performant SQL queries based on the provided input.')\
                .add_message("### Table schema:")\
                .add_schema_template(conn)\
                .add_message("### Examples of data")\
                .add_cell_value_referencing(conn, count=1)\
                .add_message(f"### Your task: {question}")\
                .build_prompt()
            

            output = self.__inference(prompt)
            pred_sql = find_sql(output, start_keyword='SELECT')
            
            df_gold = pd.read_sql(gold_sql, connection)
            try:
                df_pred = pd.read_sql(pred_sql, connection)
                
                span_df_soft        = table_similarity(df_pred, df_gold, mode='soft')
                span_df_flexible    = table_similarity(df_pred, df_gold, mode='flexible')
                span_gold_IN_pred   = False #
                span_pred_IN_gold   = False # Добавить проверку
                span_pred_columns   = df_pred.columns.to_list()
                span_ted            = self.__ted_compare(pred_sql, gold_sql)  
            except:
                # По определению полагаем
                span_df_soft        = .0
                span_df_flexible    = .0
                span_gold_IN_pred   = False
                span_pred_IN_gold   = False
                span_pred_columns   = []
                span_ted            = .0


            sql_span = ExtendedSqlSpan(
                    NL                 =question,
                    sql_gold           =gold_sql,
                    sql_pred           =pred_sql,
                    df_soft            =span_df_soft,
                    df_flexible        =span_df_flexible,
                    df_pred_IN_df_gold =span_pred_IN_gold,
                    df_gold_IN_df_pred =span_gold_IN_pred,
                    df_gold_columns    =df_gold.columns.to_list(),
                    df_pred_columns    =span_pred_columns,
                    TED                =span_ted
                )
            
            summary += span_df_flexible
            logger.append(sql_span)
        
        self.summary = summary
        self.queries_count = len(queries)
        self.logger = logger
        self.evaluated = True


    def accuracy(self):
        """
        Значение метрики Accuracy для последнего запуска модели
        """

        if not self.evaluated:
            raise Exception('Model was not been evaluated')
        
        return self.summary / self.queries_count
    

    def __ted_compare(self, sql1 : str, sql2 : str):
        """
        Компоратор для двух деревьев
        """
        
        try:
            exp1 = parse_one(sql1)
            exp2 = parse_one(sql2)
        except:
            return .0

        distiller = ChangeDistiller()
        _ = distiller.diff(exp1, exp2)
        return distiller._dice_coefficient(exp1, exp2)


    def TED(self):
        """
        Значение метрики Tree Edit Distance для последнего запуска модели
        """

        if not self.evaluated:
            raise Exception('Model was not been evaluated')
        
        summary = 0
        for span in self.logger:
            summary += self.__ted_compare(span.sql_pred, span.sql_gold)

        return summary / self.queries_count

## 1. SQLCoder 7b 

In [ ]:
sqlcoder = HuggingFaceModelInference('defog/sqlcoder-7b-2')
sqlcoder.evaluate(shuffle(dataset))

In [ ]:
sqlcoder.accuracy(), sqlcoder.sql_similarity(), np.mean(sqlcoder.exec_time)

In [ ]:
sqlcoder.logger

## DeepSeek coder 6.7b

In [ ]:
deepseek = HuggingFaceModelInference('deepseek-ai/deepseek-coder-6.7b-instruct')
deepseek.evaluate(shuffle(queries.as_list())[:10], conn) 

## 3. Chat2DB 7b

In [ ]:
# chat2db = HuggingFaceModelInference('Chat2DB/Chat2DB-SQL-7B')
# chat2db.evaluate(shuffle(dataset)[:20])

In [ ]:
# chat2db.accuracy(), chat2db.sql_similarity(), np.mean(chat2db.exec_time)

In [ ]:
# dump_inference('Chat2DB-SQL-7B', chat2db.exec_time, chat2db.sql_similarity(), chat2db.accuracy())

## 5. DuckDB-NSQL 7b

In [ ]:
# duckdb = HuggingFaceModelInference('motherduckdb/DuckDB-NSQL-7B-v0.1')

In [ ]:
# duckdb.evaluate(shuffle(dataset)[:30])

In [ ]:
# duckdb.accuracy(), duckdb.sql_similarity(), np.mean(duckdb.exec_time)

## Прочее

In [ ]:
from numba import cuda
import gc
cuda.devices.gpus[0].reset()
cuda.devices.gpus[1].reset()
gc.collect()